# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-19 23:57:01] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-19 23:57:01] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-19 23:57:01] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-19 23:57:16] INFO server_args.py:1669: Attention backend not specified. Use fa3 backend by default.


[2026-01-19 23:57:16] INFO server_args.py:2568: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=80 avail_mem=74.63 GB):  25%|██▌       | 5/20 [00:01<00:02,  6.68it/s]

Capturing batches (bs=64 avail_mem=76.79 GB):  35%|███▌      | 7/20 [00:01<00:01,  7.91it/s]

Capturing batches (bs=32 avail_mem=76.77 GB):  55%|█████▌    | 11/20 [00:01<00:00, 10.06it/s]

Capturing batches (bs=8 avail_mem=76.76 GB):  75%|███████▌  | 15/20 [00:01<00:00, 12.53it/s] 

Capturing batches (bs=1 avail_mem=76.74 GB): 100%|██████████| 20/20 [00:01<00:00, 10.31it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Joe. I was born on the 18th of December 1962, in Cleveland, Ohio. My father is Frank and my mother is Ruth. I have a younger brother named Derek and a younger sister named Laura. I have been a student for the last 3 years. I am a junior at Mount Vernon High School. I also attend the Eastern State College of Preparatory Studies. I have been going to college for almost a year now.
I have been a pro athlete since I was 16 years old. I have played hockey at various levels of the league. I have also been a member of
Prompt: The president of the United States is
Generated text:  trying to finalize a new executive order. He has a long, long list of proposals he is considering, but some are listed with punctuation marks, like this: President of the United States proposes executive order; outlining the proposal; list of proposals is long; long list includes the following: a long proposal, a complex proposal, a great proposal. The order should be comple

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [Age] year old [Gender] who is [Occupation]. I'm a [Skill/Ability] who has been [Achievement/Interest] for [Number of Years]. I'm always [Positive/Neutral/Negative] about [Current Issue/Challenge]. I'm [Personality Type] and I'm [Current Location]. I'm [Current Activity/Interest/Goal]. I'm [Current State/Emotion]. I'm [Current Goal/Goal]. I'm [Current State/Emotion]. I'm [Current Goal/Goal]. I'm [Current State/Emotion].

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light. It is a bustling metropolis with a rich history and culture, known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and Louvre Museum. Paris is also a major center for fashion, art, and cuisine, and is home to many world-renowned museums, theaters, and other attractions. The city is known for its annual festivals and events, including the Eiffel Tower Parade and the Louvre Festival. Paris is a popular tourist destination, with millions of visitors annually, making it one of the most visited cities in the world. The city is

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. Some possible future trends in AI include:

1. Increased use of AI in healthcare: AI is already being used in healthcare to diagnose and treat diseases, and it has the potential to revolutionize the field in the future.

2. Increased use of AI in finance: AI is already being used in finance to automate trading, fraud detection, and risk management. In the future, we may see even more advanced AI technologies being used in finance to improve efficiency and reduce costs.

3. Increased use of AI in transportation: AI is already



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I’m a [Type of Skill or Experience] [Skill/Experience]. I’m currently working as a [job title] and have been pursuing this career for [number of years] years. My [Highest Level of Education or Degree] is [Degree], and I am a [Specialization] expert in [Skill/Experience]. I’m passionate about [Interest or Hobby] because it’s something I enjoy doing and it makes me feel fulfilled. I also have a [Comfortable Activity or Hobby] that I enjoy doing, such as [Activity]. I’m an [Age] year old [Gender] who lives

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city where the Eiffel Tower, the Louvre Museum, and many other landmarks are located. Paris is known for its rich history, stunning architecture, and lively culture, making it one of the most popular and iconic cities in the world. It is al

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

],

...

uate

 or

 promote

 certain

 biases

In [6]:
llm.shutdown()